In [ ]:
# @REMOVE-FROM-TEMPLATE
from plaster.tools.ipynb_helpers.displays import restart_kernel; restart_kernel()

In [1]:
# @IMPORT-MERGE
import numpy as np
import pandas as pd
import itertools
from plaster.tools.log.log import error, debug
from plaster.run.job import JobResult
from plaster.run.run import RunResult
from plaster.run.plots import plots_dev
from plaster.run.sigproc_v2 import sigproc_v2_worker as worker
from plaster.tools.image.coord import WH, XY, roi_shift, clip2d
from plaster.tools.zplots import zplots
from plaster.tools.schema import check
from plaster.tools.image import imops
from plaster.tools.zap import zap
from plaster.tools.utils import utils
from plaster.tools.calibration.calibration import Calibration
from plaster.run.sigproc_v2.sigproc_v2_params import SigprocV2Params
z = zplots.setup()

Matplotlib created a temporary config/cache directory at /root/.config/matplotlib because the default path (/tmp/matplotlib-vjm9d3lz) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Loading BokehJS ...

In [2]:
# @REMOVE-FROM-TEMPLATE
from plumbum import local
job = JobResult("/erisyon/internal/jobs_folder/val_calib")
run = job.runs[0]

In [3]:
# TODO: Multichannel
z_reg_psf = np.array(run.sigproc_v2.calib["regional_psf_zstack.instrument_channel[0]"])

In [8]:
run.sigproc_v2.calib.save("val.calib")

Calibration({'regional_bg_mean.instrument_channel[0]': [[144.5614664067968, 146.8884189535546, 147.04384215478117, 146.7619119209768, 146.62870118151906], [146.28248684232554, 148.38199811899955, 148.19392501595695, 148.46578917558173, 147.01896193168523], [147.0426730477607, 147.91388571167178, 148.49503803305782, 148.89841722261698, 147.27314763149633], [145.59511503613527, 146.57221079403882, 146.27589014850895, 146.09610489744165, 144.56932114829868], [143.131371159688, 142.7073212753253, 142.52187417072088, 142.53384635805884, 142.0645086188027]], 'regional_bg_std.instrument_channel[0]': [[12.324041310059464, 12.71231768832886, 12.704092817671391, 12.52280595165263, 12.654684554382918], [12.703403079853322, 12.93341675461727, 12.989501208717996, 13.10283191828214, 12.68889252608168], [12.822280146508467, 12.728254995837741, 12.775998491270816, 12.951108649907855, 12.790967363575458], [12.62230392087801, 12.735848152214581, 12.377560527670962, 12.363561338889374, 12.294486227239243

In [4]:
z.im(np.array(run.sigproc_v2.calib["regional_illumination_balance.instrument_channel[0]"]))

In [5]:
import math
n_src_zslices = 61
n_dst_zslices = 13

dst_z_per_src_z = n_src_zslices / n_dst_zslices
print(dst_z_per_src_z)
z_src_i_best_focus = 15

# 0 0
# 1 10
# 2 20
# 3 30
# 4 40


for z_dst_i in range(n_dst_zslices):
    z_src_i0 = math.floor( 0.5 + ((z_dst_i-0.5) - n_dst_zslices//2) * dst_z_per_src_z + z_src_i_best_focus )
    z_src_i1 = math.floor( 0.5 + ((z_dst_i+0.5) - n_dst_zslices//2) * dst_z_per_src_z + z_src_i_best_focus )
    print(z_dst_i, z_src_i0, z_src_i1)



4.6923076923076925
0 -15 -11
1 -11 -6
2 -6 -1
3 -1 3
4 3 8
5 8 13
6 13 17
7 17 22
8 22 27
9 27 31
10 31 36
11 36 41
12 41 46


In [6]:
psf_im = z_reg_psf[5, :, :, :, :]
plots_dev.plot_psfs(psf_im, _size=300)

In [ ]:
cen = z_reg_psf.shape[-1] // 2
psf_im = np.moveaxis(z_reg_psf[1:12, :, :, cen, :], (0), (2))
plots_dev.plot_psfs(psf_im, _size=600)

In [ ]:
for zi in range(13):
    plots_dev.plot_psfs(z_reg_psf[zi], _size=300, f_title=f"zi={zi}")

In [ ]:
z.im(z_reg_psf[12, 2, 2])